# Two Input Networks Using Categorical Embeddings, Shared Layers, and Merge Layers

In this chapter, you will build two-input networks that use categorical embeddings to represent high-cardinality data, shared layers to specify re-usable building blocks, and merge layers to join multiple inputs to a single output. By the end of this chapter, you will have the foundational building blocks for designing neural networks with complex data flows.

# (1) Category embeddings

## Category embeddings
- Input: integers
- Output: floats
- Note: Increased dimensionality: output layer flattens back to 2D

<img src="image/Screenshot 2021-01-31 175757.png">

## Inputs

In [ ]:
input_tensor = Input(shape=(1,))

## Embedding Layer

In [ ]:
from keras.layers import Embedding
input_tensor = Input(shape=(1,))
n_team = 10887
embed_layer = Embedding(input_dim=n_team,
                        input_length=1,
                        output_dim=1,
                        name='Team-Strength-Lookup')
embed_tensor = embed_layer(input_tensor)

## Flattening

In [ ]:
from keras.layers import Flatten
flatten_tensor = Flatten()(embed_tensor)

## Put it all together

In [ ]:
input_tensor = Input(shape=(1,))
n_teams = 10887
embed_layer = Embedding(input_dim=n_teams,
                        input_length=1,
                        output_dim=1,
                        name='Team-Strength-Lookup')
embed_tensor = embed_layer(input_tensor)
flatten_tensor = Flatten()(embed_tensor)
model = Model(input_tensor, flatten_tensor)

# Exercise I: Define team lookup

Shared layers allow a model to use the same weight matrix for multiple steps. In this exercise, you will build a "team strength" layer that represents each team by a single number. You will use this number for both teams in the model. The model will learn a number for each team that works well both when the team is `team_1` and when the team is `team_2` in the input data.

The `games_season` DataFrame is available in your workspace.

### Instructions

- Count the number of unique teams.
- Create an embedding layer that maps each team ID to a single number representing that team's strength.
- The output shape should be 1 dimension (as we want to represent the teams by a single number).
- The input length should be 1 dimension (as each team is represented by exactly one id).


In [ ]:
# Imports
from keras.layers import Embedding
from numpy import unique

# Count the unique number of teams
n_teams = unique(games_season['team_1']).shape[0]

# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

# Exercise II: Define team model

The team strength lookup has three components: an input, an embedding layer, and a flatten layer that creates the output.

If you wrap these three layers in a model with an input and output, you can re-use that stack of three layers at multiple places.

Note again that the weights for all three layers will be shared everywhere we use them.

### Instructions

- Create a 1D input layer for the team ID (which will be an integer). Be sure to set the correct input shape!
- Pass this input to the team strength lookup layer you created previously.
- Flatten the output of the team strength lookup.
- Create a model that uses the 1D input as input and flattened team strength as output.


In [ ]:
# Imports
from keras.layers import Input, Embedding, Flatten
from keras.models import Model

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

# (2) Shared layers

## Shared layers
- Require the functional API
- Very flexible

<img src="image/Screenshot 2021-01-31 182054.png">

In [ ]:
input_tensor_1 = Input((1,))
input_tensor_2 = Input((1,))

In [ ]:
shared_layer = Dense(1)
output_tensor_1 = shared_layer(input_tensor_1)
output_tensor_2 = shared_layer(input_tensor_2)

## Sharing multiple layers as a model

In [ ]:
input_tensor = Input(shape=(1,))
n_teams = 10887
embed_layer = Embedding(input_dim=n_teams,
                        input_length=1,
                        output_dim=1,
                        name='Team-Strenth-Lookup')
embed_tensor = embed_layer(input_tensor)
flatten_tensor = Flatten()(embed_tensor)
model = Model(input_tensor, flatten_tensor)

In [ ]:
input_tensor_1 = Input((1,))
input_tensor_2 = Input((1,))
output_tensor_1 = model(input_tensor_1)
output_tensor_2 = model(input_tensor_2)

## Sharing multiple layers as a model

<img src="image/Screenshot 2021-01-31 182740.png">

# Exercise III: Defining two inputs

In this exercise, you will define two input layers for the two teams in your model. This allows you to specify later in the model how the data from each team will be used differently.

### Instructions

- Create an input layer to use for team 1. Recall that our input dimension is 1.
- Name the input "Team-1-In" so you can later distinguish it from team 2.
- Create an input layer to use for team 2, named "Team-2-In".


In [ ]:
# Load the input layer from keras.layers
from keras.layers import Input

# Input layer for team 1
team_in_1 = Input((1,), name='Team-1-In')

# Separate input layer for team 2
team_in_2 = Input((1,), name='Team-2-In')

# Exercise IV: Lookup both inputs in the same model

Now that you have a team strength model and an input layer for each team, you can lookup the team inputs in the shared team strength model. The two inputs will share the same weights.

In this dataset, you have 10,888 unique teams. You want to learn a strength rating for each team, such that if any pair of teams plays each other, you can predict the score, even if those two teams have never played before. Furthermore, you want the strength rating to be the same, regardless of whether the team is the home team or the away team.

To achieve this, you use a shared layer, defined by the re-usable model (`team_strength_model()`) you built in exercise 3 and the two input layers (`team_in_1` and `team_in_2`) from the previous exercise, all of which are available in your workspace.

### Instructions

- Lookup the first team ID in the team strength model.
- Lookup the second team ID in the team strength model.

In [ ]:
# Lookup team 1 in the team strength model
team_1_strength = team_strength_model(team_in_1)

# Lookup team 2 in the team strength model
team_2_strength = team_strength_model(team_in_2)